## Step 1: Install Required Libraries
First, you need to install the necessary Python libraries:

In [1]:
!pip install yt-dlp pytube pandas webvtt-py requests


Defaulting to user installation because normal site-packages is not writeable


In [11]:
import os
import requests
import pandas as pd
import webvtt
from pytube import Playlist
import yt_dlp
import re
import time
import unicodedata

# Assume `playlist` is a pytube Playlist object already initialized
playlist_url = 'https://www.youtube.com/playlist?list=PLkKDSXRppVa4i5Ib3RB6rMgUvvPdgKr0R'
playlist = Playlist(playlist_url)

# Get all video URLs in the playlist
video_urls = [video.watch_url for video in playlist.videos]

# Directory to save thumbnails and transcripts
output_dir = 'data/de_tagesschau'
os.makedirs(output_dir, exist_ok=True)

def sanitize_filename(name):
    """
    Sanitize the filename by removing invalid characters, full-width symbols,
    stripping newlines, and normalizing unicode to ASCII where possible.
    """
    # Normalize full-width characters to their ASCII equivalents
    name = unicodedata.normalize('NFKC', name)  # Normalize Unicode characters to ASCII-compatible form
    
    # Remove problematic characters for filenames
    name = re.sub(r'[\\/*?:"<>|]', "", name)  # Remove invalid characters
    name = name.replace('\n', '').replace('\r', '')  # Remove newlines
    name = name.strip()  # Trim leading/trailing spaces

    # Ensure the filename length is not too long (you can set a specific limit)
    max_filename_length = 255
    return name[:max_filename_length]

def download_thumbnail(thumbnail_url, title):
    try:
        response = requests.get(thumbnail_url)
        sanitized_title = sanitize_filename(title)
        thumbnail_file = os.path.join(output_dir, f'{sanitized_title}.jpg')
        with open(thumbnail_file, 'wb') as file:
            file.write(response.content)
        return thumbnail_file
    except Exception as e:
        print(f"Failed to download thumbnail for {title}: {e}")
        return None

def download_subtitles(video_url, video_title, subtitle_languages):
    sanitized_title = sanitize_filename(video_title)
    
    # Define the download options
    ydl_opts = {
        'writesubtitles': True,
        'writeautomaticsub': True,
        'skip_download': True,  # Skip video download
        'outtmpl': os.path.join(output_dir, f'{sanitized_title}.%(ext)s'),  # Use sanitized title
        'restrictfilenames': True,
    }
    
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            if 'de' in subtitle_languages:
                # Download English subtitles
                subtitle_file = os.path.join(output_dir, f'{sanitized_title}.de.vtt')
                if not os.path.exists(subtitle_file):
                    ydl_opts['subtitleslangs'] = ['de']
                    ydl.download([video_url])  # Download subtitles for the video
                return subtitle_file
            elif subtitle_languages:
                # Download the first available subtitle if English is not found
                first_lang = list(subtitle_languages)[0]
                subtitle_file = os.path.join(output_dir, f'{sanitized_title}.{first_lang}.vtt')
                if not os.path.exists(subtitle_file):
                    ydl_opts['subtitleslangs'] = [first_lang]
                    ydl.download([video_url])
                return subtitle_file
                
    except yt_dlp.utils.DownloadError as e:
        error_message = str(e)
        print(f"Error downloading subtitles for {video_title}: {error_message}")
        return None
        
def download_video_info(video_url, max_retries=3, retry_interval=180):
    ydl_opts = {
        'writesubtitles': True,
        'writeautomaticsub': True,
        'skip_download': True,
        #'outtmpl': os.path.join(output_dir, f'{"%(sanitized_title)s"}.%(ext)s'),
        'postprocessors': [{
            'key': 'FFmpegMetadata',
        }],
    }
    retries = 0
    while retries < max_retries:
        try:
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                info_dict = ydl.extract_info(video_url, download=False)
                video_title = info_dict.get('title', None)
                video_id = info_dict.get('id', None)
                thumbnail_url = info_dict.get('thumbnail', None)
                publish_date = info_dict.get('upload_date', None)  # Format: 'YYYYMMDD'
                subtitle_languages = info_dict.get('automatic_captions', {}).keys()
                description = info_dict.get('description', None)

                # Download thumbnail
                thumbnail_file = None
                if thumbnail_url:
                    thumbnail_file = download_thumbnail(thumbnail_url, video_title)

                # Download subtitles
                subtitle_file = None
                subtitle_file = download_subtitles(video_url, video_title, subtitle_languages)
                
            # Return the collected video information
            return video_id, video_title, thumbnail_file, subtitle_file, publish_date, description

        except yt_dlp.utils.DownloadError as e:
            error_message = str(e)
            if "Sign in to confirm you’re not a bot" in error_message:
                retries += 1
                print(f"Error: {error_message}. Retrying in {retry_interval} seconds... (Attempt {retries}/{max_retries})")
                time.sleep(retry_interval)  # Wait before retrying
            else:
                print(f"Failed to download video info for {video_url}: {e}")
                return None, None, None, None, None, None

    # After max retries, log the failure and return None
    print(f"Max retries reached. Skipping video: {video_url}")
    return None, None, None, None, None, None

def vtt_to_text(vtt_file_path):
    try:
        text = ''
        lines = []
        deduplicated_lines = []
        for caption in webvtt.read(vtt_file_path):
            # Split the block into lines and deduplicate them within the block
            lines.extend(caption.text.strip().split('\n'))
            #print(caption.text.strip().split('\n'))
        for line in lines:
            if line and (not deduplicated_lines or line != deduplicated_lines[-1]):  # Avoid consecutive duplicates
                deduplicated_lines.append(line)
     
        text = "\n".join(deduplicated_lines)
    
    except Exception as e:
        print(f"Failed to convert {vtt_file_path} to text: {e}")
        
    return text

# Create a list to store video information
data = []

# Loop through each video URL and download the required data
for video_url in video_urls:
    video_id, video_title, thumbnail_file, subtitle_file, publish_date, description = download_video_info(video_url)
    # If video info is successfully retrieved
    if video_id and video_title:
        # Convert subtitles to text if available
        transcript = None
        if subtitle_file and os.path.exists(subtitle_file):
            transcript = vtt_to_text(subtitle_file)
        else:
            print("subtitle file not found")
        # Append data to the list
        data.append({
            'video_id': video_id,
            'video_title': video_title,
            'thumbnail_path': thumbnail_file,
            'transcript': transcript,
            'publish_date': publish_date,
            'description': description
        })

# Convert the list to a DataFrame
df = pd.DataFrame(data, columns=['video_id', 'video_title', 'thumbnail_path', 'transcript', 'publish_date','description'])

# Save the DataFrame to a CSV file
df.to_csv(os.path.join(output_dir,'../', 'de_tagesschau.csv'), index=False)

print(f"Dataset created successfully with {len(df)} entries.")

[youtube] Extracting URL: https://youtube.com/watch?v=NX3VEeL3aW8
[youtube] NX3VEeL3aW8: Downloading webpage
[youtube] NX3VEeL3aW8: Downloading ios player API JSON
[youtube] NX3VEeL3aW8: Downloading web creator player API JSON
[youtube] NX3VEeL3aW8: Downloading m3u8 information


subtitle file not found
[youtube] Extracting URL: https://youtube.com/watch?v=W_16zPz9MRg
[youtube] W_16zPz9MRg: Downloading webpage
[youtube] W_16zPz9MRg: Downloading ios player API JSON
[youtube] W_16zPz9MRg: Downloading web creator player API JSON
[youtube] W_16zPz9MRg: Downloading m3u8 information


subtitle file not found
[youtube] Extracting URL: https://youtube.com/watch?v=eE32NFyT-xE
[youtube] eE32NFyT-xE: Downloading webpage
[youtube] eE32NFyT-xE: Downloading ios player API JSON
[youtube] eE32NFyT-xE: Downloading web creator player API JSON
[youtube] eE32NFyT-xE: Downloading m3u8 information


subtitle file not found
[youtube] Extracting URL: https://youtube.com/watch?v=jasQxm52eLI
[youtube] jasQxm52eLI: Downloading webpage
[youtube] jasQxm52eLI: Downloading ios player API JSON
[youtube] jasQxm52eLI: Downloading web creator player API JSON
[youtube] jasQxm52eLI: Downloading m3u8 information


subtitle file not found
[youtube] Extracting URL: https://youtube.com/watch?v=6UdGy-kNREs
[youtube] 6UdGy-kNREs: Downloading webpage
[youtube] 6UdGy-kNREs: Downloading ios player API JSON
[youtube] 6UdGy-kNREs: Downloading web creator player API JSON
[youtube] 6UdGy-kNREs: Downloading m3u8 information


...

subtitle file not found
[youtube] Extracting URL: https://youtube.com/watch?v=pGX7E5gnnNo
[youtube] pGX7E5gnnNo: Downloading webpage
[youtube] pGX7E5gnnNo: Downloading ios player API JSON
[youtube] pGX7E5gnnNo: Downloading web creator player API JSON
[youtube] pGX7E5gnnNo: Downloading m3u8 information


subtitle file not found
[youtube] Extracting URL: https://youtube.com/watch?v=xWKJ7YB-xqI
[youtube] xWKJ7YB-xqI: Downloading webpage
[youtube] xWKJ7YB-xqI: Downloading ios player API JSON
[youtube] xWKJ7YB-xqI: Downloading web creator player API JSON
[youtube] xWKJ7YB-xqI: Downloading m3u8 information


subtitle file not found
[youtube] Extracting URL: https://youtube.com/watch?v=vXm4noih1cY
[youtube] vXm4noih1cY: Downloading webpage
[youtube] vXm4noih1cY: Downloading ios player API JSON
[youtube] vXm4noih1cY: Downloading web creator player API JSON
[youtube] vXm4noih1cY: Downloading m3u8 information


subtitle file not found
[youtube] Extracting URL: https://youtube.com/watch?v=3-vFYmYNjSA
[youtube] 3-vFYmYNjSA: Downloading webpage
[youtube] 3-vFYmYNjSA: Downloading ios player API JSON
[youtube] 3-vFYmYNjSA: Downloading web creator player API JSON
[youtube] 3-vFYmYNjSA: Downloading m3u8 information


subtitle file not found
Dataset created successfully with 181 entries.
